# Import modules

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import math
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GroupKFold
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from functools import partial
from scipy.optimize import minimize
from collections import defaultdict
import warnings
warnings.simplefilter(action='ignore')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def smape(y_true, y_pred):

    y_true = y_true + 1
    y_pred = y_pred + 1
    
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

In [3]:
def smape_plus_1(y_true, y_pred):
    y_true_plus_1 = y_true + 1
    y_pred_plus_1 = y_pred + 1
    metric = np.zeros(len(y_true_plus_1))
    
    numerator = np.abs(y_true_plus_1 - y_pred_plus_1)
    denominator = ((np.abs(y_true_plus_1) + np.abs(y_pred_plus_1)) / 2)
    
    mask_not_zeros = (y_true_plus_1 != 0) | (y_pred_plus_1 != 0)
    metric[mask_not_zeros] = numerator[mask_not_zeros] / denominator[mask_not_zeros]
    
    return 100 * np.nanmean(metric)

# Reading Data

In [4]:
# Load a dataset into a Pandas DataFrame
train_clinical = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv')
proteins = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv')
proteins_features = pd.pivot_table(proteins, values='NPX', index='visit_id', columns='UniProt', aggfunc='sum')
train_clinical = train_clinical[~train_clinical.visit_month.isin([3,5,9])].copy()

train_clinical = train_clinical.merge(
    proteins_features,
    left_on='visit_id',
    right_index=True,
    how='left'
)

In [5]:
# Filling missing values of NPX
train_clinical[proteins_features.columns] = train_clinical.groupby('patient_id')[proteins_features.columns].\
                                                    fillna(train_clinical.groupby('patient_id')[proteins_features.columns].transform('mean'))

In [6]:
train_clinical['pred_month'] = train_clinical['visit_month']

for plus_month in [6, 12, 24]:
    train_shift = train_clinical[['patient_id', 'visit_month', 'pred_month', 'updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']].copy()
    train_shift['visit_month'] -= plus_month
    train_shift.rename(columns={f'updrs_{i}': f'updrs_{i}_plus_{plus_month}' for i in range(1, 5)}, inplace=True)
    train_shift.rename(columns={'pred_month': f'pred_month_plus_{plus_month}'}, inplace=True)
    train_clinical = train_clinical.merge(train_shift, how='left', on=['patient_id', 'visit_month'])

train_clinical.rename(columns={f'updrs_{i}': f'updrs_{i}_plus_0' for i in range(1, 5)}, inplace=True)
train_clinical.rename(columns={'pred_month': f'pred_month_plus_0'}, inplace=True)

# Training

In [7]:
def calculate_predictions(pred_month, trend, target):
    if target == 'updrs_4': pred_month = pred_month.clip(54, None)
    if len(trend) == 2:
        return np.round(trend[0] + pred_month * trend[1]) # linear prediction
    return np.round(trend[0] + pred_month * trend[1] + np.square(pred_month) * trend[2]) # quadratic prediction


def calculate_predicitons_protein(protein, pred_month, protein_shift):
    trend_pred_month = target_to_trend[u]
    pred_month_trend = calculate_predictions(pred_month=pred_month, trend=trend_pred_month, target=target)
    return np.round(pred_month_trend + protein_shift)

In [8]:
def function_to_minimize_model(x, y_true_array_tr, pred_month_array_tr):    
    metric = smape_plus_1(
        y_true=y_true_array_tr, 
        y_pred=calculate_predictions(
            pred_month=pred_month_array_tr,
            trend=x,
            target=target
        )
    )
    return metric

def function_to_minimize(x):
    metric = smape_plus_1(
        y_true=y_true_array, 
        y_pred=calculate_predicitons_protein(
            protein=protein_array,
            pred_month=pred_month_array,
            protein_shift=x[0]
        )
    )
    return metric

In [9]:
# To find Best Shift for NPX
def find_best_const(train_clinical_all_filtered, target):
    columns_with_target = [f'{target}_plus_{plus_month}' for plus_month in [0, 6, 12, 24]]
    columns_with_pred_month = [f'pred_month_plus_{plus_month}' for plus_month in [0, 6, 12, 24]]
    global y_true_array
    global pred_month_array
    global protein_array
    y_true_array = train_clinical_all_filtered[columns_with_target].values.ravel()
    pred_month_array = train_clinical_all_filtered[columns_with_pred_month].values.ravel()
    protein_array = np.concatenate([train_clinical_all_filtered[feature].values] * 4)
    result = minimize(
        fun=function_to_minimize,
        x0=[0.0],
        method='Powell'
    ).x[0]
    return result

In [10]:
def model(y_true_array_tr, pred_month_array_tr):
    """Fits a linear or quadratic model to the given data"""
    return list(minimize(
        fun=partial(function_to_minimize_model,
                    y_true_array_tr=y_true_array_tr,
                    pred_month_array_tr=pred_month_array_tr
                   ),
        # if x0 has two elements, the predictions will be linear
        # if x0 has three elements, the predictions will be quadratic
#         x0=[0, 0] if target != 'updrs_2' else [0,0],
        x0=[0, 0],
        method='Powell'
    ).x)

In [11]:
# feature = 'Q9UNU6'
npx_groups = [
    {'quantile_low': 0.0, 'quantile_high': 0.05},
    {'quantile_low': 0.05, 'quantile_high': 0.95},
    {'quantile_low': 0.95, 'quantile_high': 1.0},
]
def get_shifts(data):
    target_to_npx_groups_shift = defaultdict(list)
    for target in ['updrs_1', 'updrs_2', 'updrs_3']:
        for npx_group in npx_groups:
            item = npx_group.copy()
            item['feature'] = feature

            if item['quantile_low'] == 0:
                item['quantile_low_value'] = -np.inf
            else:
                item['quantile_low_value'] = data[feature].quantile(item['quantile_low'])

            if item['quantile_high'] == 1:
                item['quantile_high_value'] = np.inf
            else: 
                item['quantile_high_value'] = data[feature].quantile(item['quantile_high'])

            data = data[
                (data[feature] >= item['quantile_low_value'])
                & (data[feature] < item['quantile_high_value'])
            ]

            item['shift'] = find_best_const(data, target)
            target_to_npx_groups_shift[target].append(item)
    return target_to_npx_groups_shift


# Validation

In [12]:
target = ['updrs_4','updrs_3','updrs_2','updrs_1']
FEATURES = ['visit_month']
smapes = []

for u in target:
    print(u,':')
    
    scores = [] 
    columns_with_target = [f'{u}_plus_{plus_month}' for plus_month in [0, 6, 12, 24]]
    columns_with_pred_month = [f'pred_month_plus_{plus_month}' for plus_month in [0, 6, 12, 24]]
    
    for train_index, test_index in GroupKFold(n_splits=5).split(train_clinical, groups=train_clinical['patient_id']):
        
        # The model
        y_true_array_tr = train_clinical.iloc[train_index][columns_with_target].values.ravel()
        pred_month_array_tr = train_clinical.iloc[train_index][columns_with_pred_month].values.ravel()

        trend = model(y_true_array_tr, pred_month_array_tr)

        y_true_array_va = train_clinical.iloc[test_index][columns_with_target].values.ravel()
        pred_month_array_va = train_clinical.iloc[test_index][columns_with_pred_month].values.ravel()
        
        y_pred = pd.Series(calculate_predictions(pred_month_array_va, trend, u))
        
        
        # The Shifting
        for feature in ['Q9UNU6','P54289','P16152','P14174','P08133','P01877','P01608',
                        'P00736','P01031','P01717','P01833','P01861','P02652','P02679',
                        'P02749','P02765','P02766','P04433','P05408','P08133','P20933']:
            target_to_trend = {}
            target_to_trend[u] = trend
            target_to_npx_groups_shift = get_shifts(train_clinical.loc[train_index,:])
            for item in target_to_npx_groups_shift[u]:
                feature = item['feature']
                mask_feature_range = (
                    (train_clinical[feature] >= item['quantile_low_value'])
                    & (train_clinical[feature] < item['quantile_high_value'])
                )
                y_pred.loc[mask_feature_range] += item['shift'] / 21
        
        
        score = smape_plus_1(y_true_array_va, np.round(y_pred.values))
        scores.append(score)
#         print(scores[-1])           
    
    
    print("Mean:",np.mean(scores),"\nSTD: ", np.std(scores),'\n')
    smapes.append(np.mean(scores))



# updrs_4 :
# Mean: 49.486072107273785 
# STD:  11.924699408010685 

# updrs_3 :
# Mean: 67.72529162773566 
# STD:  6.392222675072084 

# updrs_2 :
# Mean: 70.97234993332395 
# STD:  2.1967032826089397 

# updrs_1 :
# Mean: 54.46640125576296 
# STD:  5.219863520315581 

updrs_4 :
Mean: 50.482361114306215 
STD:  12.391242490479586 

updrs_3 :
Mean: 68.30175241859689 
STD:  6.45304860151447 

updrs_2 :
Mean: 70.9263346726284 
STD:  2.0353709337444736 

updrs_1 :
Mean: 54.555168524621344 
STD:  5.349233761491058 



In [13]:
print(f'Average for the 4 targets: {np.mean(smapes)}')
# No shift: 61.229836793527554
# Mean + Shift: 62.7248326879205
# Mean + Top 7: 61.026151387996784
# Mean + Top 21: 61.01899590753103
# Mean + Top 21 + Weight 15 + No updrs_3: 60.99082925276876
# Mean + Top 21 + quadratic for all except 4: 60.66252873102408

Average for the 4 targets: 61.066404182538214


# Training For All Data

In [14]:
# # Retrain with full data
# target_to_trend = {}
# for i in range(1, 5):
#     target = f'updrs_{i}'
#     columns_with_target = [f'{target}_plus_{plus_month}' for plus_month in [0, 6, 12, 24]]
#     columns_with_pred_month = [f'pred_month_plus_{plus_month}' for plus_month in [0, 6, 12, 24]]
#     y_true_array = train_clinical[columns_with_target].values.ravel()
#     pred_month_array = train_clinical[columns_with_pred_month].values.ravel()
#     trend = model(y_true_array, pred_month_array)
#     target_to_trend[target] = trend
# display(target_to_trend)



# Shifting For All Data

In [15]:
def calculate_predicitons_protein(protein, pred_month, protein_shift):
    trend_pred_month = target_to_trend[target]
    pred_month_trend = calculate_predictions(pred_month=pred_month, trend=trend_pred_month, target=target)
    return np.round(pred_month_trend + protein_shift)

# Inference

In [16]:
import amp_pd_peptide_310
amp_pd_peptide_310.make_env.func_dict['__called__'] = False
env = amp_pd_peptide_310.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test files

proteins_features_all = pd.DataFrame()

# The API will deliver four dataframes in this specific order:
for test_clinical_data, test_peptides, test_proteins, sample_submission in iter_test:
    sample_submission['patient_id'] = sample_submission['prediction_id'].map(lambda x: int(x.split('_')[0]))
    sample_submission['visit_month'] = sample_submission['prediction_id'].map(lambda x: int(x.split('_')[1]))
    sample_submission['target_name'] = sample_submission['prediction_id'].map(lambda x: 'updrs_' + x.split('_')[3])
    sample_submission['plus_month'] = sample_submission['prediction_id'].map(lambda x: int(x.split('_')[5]))
    sample_submission['pred_month'] = sample_submission['visit_month'] + sample_submission['plus_month']
    sample_submission['visit_id'] = sample_submission['patient_id'].astype(str) + '_' + sample_submission['visit_month'].astype(str)
    
    proteins_features = pd.pivot_table(test_proteins, values='NPX', index='visit_id', columns='UniProt', aggfunc='sum')
    proteins_features['visit_id'] = proteins_features.index
    proteins_features_all = pd.concat([proteins_features_all, proteins_features])
    proteins_features_all['patient_id'] = proteins_features_all.index.map(lambda x: int(x.split('_')[0]))
    proteins_features_all[proteins_features.columns] = proteins_features_all.groupby('patient_id')[proteins_features.columns].\
                                                                                                   fillna(proteins_features_all.groupby('patient_id')[proteins_features.columns].transform('mean'))
    proteins_features = proteins_features_all.groupby('patient_id', as_index=False).last()
    
    sample_submission = sample_submission.merge(
        proteins_features,
        on='patient_id',
        how='left'
    )

    for i in range(1, 5):
        
        target_to_trend = {'updrs_1': [6.870713324069481, 2.064017287846742e-11],
                           'updrs_2': [7.2605144568170275, 0.005820570100281764],
                           'updrs_3': [19.429459027194152, 0.14400963598833913],
                           'updrs_4': [3.5531891360188255e-11, 0.0070438433277029925]}
        
        
        # Model Predictions
        target = f'updrs_{i}'
        mask_target = sample_submission['target_name'] == target
        sample_submission.loc[mask_target, 'rating'] = calculate_predictions(
            pred_month=sample_submission.loc[mask_target, 'pred_month'],
            trend=target_to_trend[target],
            target = target
        )
        
            
        # The Shifting
        weight = 21
        for feature in ['Q9UNU6','P54289','P16152','P14174','P08133','P01877','P01608',
                        'P00736','P01031','P01717','P01833','P01861','P02652','P02679',
                        'P02749','P02765','P02766','P04433','P05408','P08133','P20933']:
            try:
                target_to_trend = {}
                target_to_trend[target] = trend
                target_to_npx_groups_shift = get_shifts(train_clinical)
                for item in target_to_npx_groups_shift[target]:
                    feature = item['feature']
                    mask_feature_range = mask_target & (
                        (sample_submission[feature] >= item['quantile_low_value'])
                        & (sample_submission[feature] < item['quantile_high_value'])
                    )
                    sample_submission.loc[mask_feature_range, 'rating'] += item['shift'] / weight
            except:
                weight -= 1
                continue
        sample_submission.loc[mask_target, 'rating'] = np.round(sample_submission.loc[mask_target, 'rating'])
        
    # call env.predict for every iteration
    env.predict(sample_submission[['prediction_id', 'rating']])

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


In [17]:
sample_submission[['prediction_id', 'rating']]

,prediction_id,rating
0,3342_6_updrs_1_plus_0_months,6.0
1,3342_6_updrs_1_plus_6_months,6.0
2,3342_6_updrs_1_plus_12_months,6.0
3,3342_6_updrs_1_plus_24_months,6.0
4,3342_6_updrs_2_plus_0_months,7.0
5,3342_6_updrs_2_plus_6_months,7.0
6,3342_6_updrs_2_plus_12_months,7.0
7,3342_6_updrs_2_plus_24_months,7.0
8,3342_6_updrs_3_plus_0_months,20.0
9,3342_6_updrs_3_plus_6_months,21.0
